In [15]:
import json
import os

import boto3
import requests
from botocore.auth import SigV4Auth
from botocore.awsrequest import AWSRequest
from dotenv import load_dotenv
from fastmcp import Client
from rich.pretty import pprint

load_dotenv()

True

### Let's define MCP Tools

In [16]:
mcp_client = Client("02_agent_mcp_server.py")
await mcp_client._connect()

mcp_tools = await mcp_client.list_tools()

tools = [
        {
            "name": tool.name,
            "description": tool.description,
            "input_schema": getattr(tool, 'inputSchema', {"type": "object", "properties": {}}),
        }
        for tool in mcp_tools
    ]
# pprint(mcp_tools, expand_all=True)
pprint(tools, expand_all=True)

[
│   {
│   │   'name': 'get_weather',
│   │   'description': 'Get the current weather in a given location\n\nArgs:\n    latitude: Latitude of the location for which to get the weather\n    longitude: Longitude of the location for which to get the weather\n\nReturns:\n    JSON with current weather information',
│   │   'input_schema': {
│   │   │   'properties': {
│   │   │   │   'latitude': {
│   │   │   │   │   'type': 'string'
│   │   │   │   },
│   │   │   │   'longitude': {
│   │   │   │   │   'type': 'string'
│   │   │   │   }
│   │   │   },
│   │   │   'required': [
│   │   │   │   'latitude',
│   │   │   │   'longitude'
│   │   │   ],
│   │   │   'type': 'object'
│   │   }
│   },
│   {
│   │   'name': 'facts_from_wikipedia',
│   │   'description': 'Get Wikipedia facts about a given topic, location, person etc.\n\nArgs:\n    topic: A topic, location, person etc. to search for on Wikipedia\n\nReturns:\n    JSON with Wikipedia information',
│   │   'input_schema': {
│   │   │   'properties': {
│   │   │   │   'topic': {
│   │   │   │   │   'type': 'string'
│   │   │   │   }
│   │   │   },
│   │   │   'required': [
│   │   │   │   'topic'
│   │   │   ],
│   │   │   'type': 'object'
│   │   }
│   }
]

### Let's define the user's message

In [17]:
user_message = "What's the weather in Warsaw? Is there anything interesting about this city?"

### Let's make the first LLM call to decide which tool to use and with what parameters

In [18]:
region = os.getenv("AWS_REGION", "eu-central-1")
url = f"https://bedrock-runtime.{region}.amazonaws.com/model/eu.anthropic.claude-sonnet-4-5-20250929-v1:0/invoke"

llm_headers = {
    "Content-Type": "application/json",
}

In [19]:
session = boto3.Session()
credentials = session.get_credentials()

payload = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 300,
    "tools": tools,
    "tool_choice": {"type": "auto"},
    "messages": [
        {
            "role": "user",
            "content": user_message,
        }
    ]
}

request = AWSRequest(method='POST', url=url, data=json.dumps(payload), headers=llm_headers)
SigV4Auth(credentials, "bedrock", region).add_auth(request)

resp = requests.post(url, headers=dict(request.headers), data=json.dumps(payload))
response_content = resp.json().get("content", [])
response_content

[{'type': 'text',
  'text': "I'll help you get the weather information for Warsaw and some interesting facts about the city."},
 {'type': 'tool_use',
  'id': 'toolu_bdrk_01EuXFmwCXwKFT2mZeTqjEFU',
  'name': 'get_weather',
  'input': {'latitude': '52.2297', 'longitude': '21.0122'}},
 {'type': 'tool_use',
  'id': 'toolu_bdrk_0172GggaGrZmHthScTa8Ts4G',
  'name': 'facts_from_wikipedia',
  'input': {'topic': 'Warsaw'}}]

### Now let's process the output from the tool use step

In [20]:
# Extracting tools
tools_to_use = [block for block in response_content if block.get("type") == "tool_use"]
tools_to_use

[{'type': 'tool_use',
  'id': 'toolu_bdrk_01EuXFmwCXwKFT2mZeTqjEFU',
  'name': 'get_weather',
  'input': {'latitude': '52.2297', 'longitude': '21.0122'}},
 {'type': 'tool_use',
  'id': 'toolu_bdrk_0172GggaGrZmHthScTa8Ts4G',
  'name': 'facts_from_wikipedia',
  'input': {'topic': 'Warsaw'}}]

In [21]:
# Use tools
for tool in tools_to_use:
    print(tool)
    result = await mcp_client.call_tool(tool['name'], tool['input'])
    if hasattr(result, 'content') and result.content:
        tool['output'] = result.content[0].text
    else:
        tool['output'] = str(result)

{'type': 'tool_use', 'id': 'toolu_bdrk_01EuXFmwCXwKFT2mZeTqjEFU', 'name': 'get_weather', 'input': {'latitude': '52.2297', 'longitude': '21.0122'}}
{'type': 'tool_use', 'id': 'toolu_bdrk_0172GggaGrZmHthScTa8Ts4G', 'name': 'facts_from_wikipedia', 'input': {'topic': 'Warsaw'}}


In [22]:
# Collecting outputs from all tools into a single string
tool_outputs = "\n".join([f"Tool: {tool['name']}, Output: {tool.get('output', 'No output')}" for tool in tools_to_use])
print(tool_outputs)

Tool: get_weather, Output: {
  "success": true,
  "location": {
    "latitude": "52.2297",
    "longitude": "21.0122"
  },
  "weather": {
    "time": "2025-10-24T06:00",
    "interval": 900,
    "temperature": 12.1,
    "windspeed": 19.4,
    "winddirection": 294,
    "is_day": 1,
    "weathercode": 3
  }
}
Tool: facts_from_wikipedia, Output: {
  "success": true,
  "topic": "Warsaw",
  "data": {
    "batchcomplete": "",
    "warnings": {
      "main": {
        "*": "Subscribe to the mediawiki-api-announce mailing list at <https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/> for notice of API deprecations and breaking changes. Use [[Special:ApiFeatureUsage]] to see usage of deprecated features by your application."
      },
      "revisions": {
        "*": "Because \"rvslots\" was not specified, a legacy format has been used for the output. This format is deprecated, and in the future the new format will always be used."
      }
    },
    "query": 

### Finally, let's make the last LLM call to get the final answer

In [23]:
final_prompt = f"""You are an assistant that responds to user question based on the tools used. Respond politely to user's question using the output from the tools. Use Celsius scale. If the tools output is empty, then always respond with "I don't know". Use only information from the tools output to answer the question.

<user question>
{user_message}
</user question>

<tool_outputs>
{tool_outputs}
</tool_outputs>

Remember, to answer the question, use the information from the <tool_outputs> section only.
"""

In [24]:
print(final_prompt[:1000] + '.' * 50 + final_prompt[-200:])

You are an assistant that responds to user question based on the tools used. Respond politely to user's question using the output from the tools. Use Celsius scale. If the tools output is empty, then always respond with "I don't know". Use only information from the tools output to answer the question.

<user question>
What's the weather in Warsaw? Is there anything interesting about this city?
</user question>

<tool_outputs>
Tool: get_weather, Output: {
  "success": true,
  "location": {
    "latitude": "52.2297",
    "longitude": "21.0122"
  },
  "weather": {
    "time": "2025-10-24T06:00",
    "interval": 900,
    "temperature": 12.1,
    "windspeed": 19.4,
    "winddirection": 294,
    "is_day": 1,
    "weathercode": 3
  }
}
Tool: facts_from_wikipedia, Output: {
  "success": true,
  "topic": "Warsaw",
  "data": {
    "batchcomplete": "",
    "warnings": {
      "main": {
        "*": "Subscribe to the mediawiki-api-announce mailing list at <https://lists.wikimedia.org/postorius/lis

In [25]:
payload = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 300,
    "messages": [
        {
            "role": "user",
            "content": final_prompt,
        }
    ]
}

request = AWSRequest(method='POST', url=url, data=json.dumps(payload), headers=llm_headers)
SigV4Auth(credentials, "bedrock", region).add_auth(request)

final_response = requests.post(url, headers=dict(request.headers), data=json.dumps(payload))
final_response.text

'{"model":"claude-sonnet-4-5-20250929","id":"msg_bdrk_01N5NhQ24aFVYdpKE4swtDG3","type":"message","role":"assistant","content":[{"type":"text","text":"Based on the weather data and information provided:\\n\\n## Weather in Warsaw\\n\\nThe current weather in Warsaw is:\\n- **Temperature**: 12.1°C\\n- **Wind speed**: 19.4 km/h from the west-northwest (294°)\\n- **Sky conditions**: Overcast (weathercode 3)\\n- **Time of observation**: October 24, 2025 at 6:00 AM\\n- **Daytime**: Yes\\n\\n## Interesting Facts About Warsaw\\n\\nYes, there are many interesting things about Warsaw!\\n\\n**Historical Significance:**\\n- Warsaw is the capital and largest city of Poland with a population of approximately 1.86 million residents\\n- The city was almost completely destroyed during World War II - about 85% of it was razed to the ground, including the historic Old Town\\n- Despite this destruction, the Old Town was meticulously reconstructed and is now a UNESCO World Heritage Site\\n\\n**Cultural Herit

### Let's see the final response

In [26]:
print(final_response.json().get("content", [])[0].get("text", "No response"))

Based on the weather data and information provided:

## Weather in Warsaw

The current weather in Warsaw is:
- **Temperature**: 12.1°C
- **Wind speed**: 19.4 km/h from the west-northwest (294°)
- **Sky conditions**: Overcast (weathercode 3)
- **Time of observation**: October 24, 2025 at 6:00 AM
- **Daytime**: Yes

## Interesting Facts About Warsaw

Yes, there are many interesting things about Warsaw!

**Historical Significance:**
- Warsaw is the capital and largest city of Poland with a population of approximately 1.86 million residents
- The city was almost completely destroyed during World War II - about 85% of it was razed to the ground, including the historic Old Town
- Despite this destruction, the Old Town was meticulously reconstructed and is now a UNESCO World Heritage Site

**Cultural Heritage:**
- Warsaw is the birthplace of Marie Curie (Maria Skłodowska-Curie), the first female Nobel Prize winner
- The city has a rich musical heritage - Frédéric Chopin moved here as an infan

In [27]:
# Cleanup
await mcp_client._disconnect()